In [146]:
import pandas as pd
import os
import numpy as np
import sc2reader
import threading
import matplotlib.pyplot as plt

In [16]:
rep_folder = "data/s2client_blizz_replays/"
rep_names = [rep_name for root_dir, rep_dir, rep_name in os.walk(rep_folder)][0]
rep_paths = [rep_folder + rep_name for rep_name in rep_names]

In [74]:
def loadReplay(rep_path):
    return sc2reader.load_replay(rep_path, load_level=4)

In [75]:
def loadReplays(until, since = 0):
    return [loadReplay(rep_path) for rep_path in rep_paths[since:until]]

In [76]:
def getPlayersData(replay):
    player_data = [player.__dict__ for player in replay.players]
    game_length = replay.length.seconds
    return [makeDatapoint(player, game_length) for player in player_data]

In [77]:
def makeDatapoint(player, game_length):
    result = player['result']
    race = player['attribute_data']['Race']
    events = sum(1 for event in player['events'] if event.__dict__['name'] != 'CameraEvent')
    league_index = player['highest_league']
    return race, league_index, events, result, game_length, 

In [102]:
%%time
data = loadReplays(5000)

CPU times: user 13min 10s, sys: 5min 29s, total: 18min 39s
Wall time: 20min 13s


In [103]:
%%time
playerPairs = [getPlayersData(replay) for replay in data]
raw_dataset = [player for playerPair in playerPairs for player in playerPair]

In [150]:
cols = ['Race', 'LeagueIndex', 'Actions', 'GameResult', 'GameLength']
df = pd.DataFrame(raw_dataset, columns=cols)
df['APM'] = df['Actions'] / (df['GameLength'] / 60)
df = df.loc[df["GameLength"] > 60]

In [151]:
apm_for_winners = df.groupby('GameResult').agg(['median', 'mean', 'count']).reset_index()[['GameResult', 'APM']]
apm_for_winners

GameResult         APM                  
                  median        mean count
0       Loss  104.985207  119.684946  4478
1        Win  116.270365  131.122283  4478

In [152]:
apm_by_league = df.groupby('LeagueIndex').agg(['median', 'mean', 'count']).reset_index()[['LeagueIndex', 'APM']]
apm_by_league

LeagueIndex         APM                  
                   median        mean count
0           0   89.196164  105.998395  1036
1           1   46.783217   59.484513   251
2           2   66.473180   74.868218  1000
3           3   81.422679   88.800274  1146
4           4  102.076503  112.545487  1683
5           5  138.970588  147.367665  2911
6           6  206.760168  209.614918   775
7           7  250.819968  256.725394   166